## Libraries

In [2]:
import os
import time
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import seaborn as sns
from dateutil import parser
from google.cloud import bigquery
from IPython.display import display, HTML
from plotly.io import to_html
import matplotlib.pyplot as plt

In [3]:
def bq_run(query):
    query_job = client.query(query)
    return query_job.to_dataframe()

In [4]:
client = bigquery.Client()

## Average hourly load for 2 months

In [101]:
query = '''

WITH

-- get load data for the last 60 days
filtered_data AS (
  SELECT
    TIMESTAMP_TRUNC(event_publish_time, SECOND) AS event_time,
    metrics_metered_active_power_w AS p_active_load,
    metrics_metered_current_load_w AS p_current_load,
    metrics_required_min_power_w AS p_min_power_w,
    metrics_required_max_power_w AS p_max_power_w,
    ROW_NUMBER() OVER (PARTITION BY TIMESTAMP_TRUNC(event_publish_time, SECOND) ORDER BY event_publish_time) AS rn
  FROM
    `fever-data-prod.fever.control_log`
  WHERE
    event_publish_time > "2024-04-01 00:00:00.000000+00:00"
    AND event_publish_time < "2024-05-31 23:59:59.000000+00:00"
    -- S3
    AND asset_id = "98b95ea6-16e8-45de-83b1-f5676d373e8d"
),

-- sample load once per second
sampled_data AS (
  SELECT
    event_time,
    p_current_load / 1000.0 AS p_current_load_kw,
    
    --p_active_load / 1000.0 AS p_active_load_kw,
    --p_min_power_w / 1000.0 AS p_min_power_kw,
    --p_max_power_w / 1000.0 AS p_max_power_kw,
  FROM
    filtered_data
  WHERE
    rn = 1
),

-- avg load per hour
average_load_per_hour AS (
  SELECT
    TIMESTAMP_TRUNC(event_time, HOUR) AS date_hour,
    AVG(p_current_load_kw) AS avg_p_current_load_kw,
  FROM
    sampled_data
  GROUP BY
    TIMESTAMP_TRUNC(event_time, HOUR)
)

SELECT
  date_hour,
  avg_p_current_load_kw,
FROM
  average_load_per_hour al
ORDER BY
  al.date_hour DESC;


'''

query_job = client.query(query)
df_agg = query_job.to_dataframe()

In [102]:
df_agg['avg_p_current_load_mw'] = df_agg['avg_p_current_load_kw'] / 1000.0

In [ ]:
df_agg['date_hour'] = pd.to_datetime(df_agg['date_hour'])
df_agg['hour'] = df_agg['date_hour'].dt.hour
df_agg['hour_formatted'] = df_agg['hour'].apply(lambda x: f'{x:02d}:00')

fig = px.box(df_agg, x='hour_formatted', y='avg_p_current_load_mw', title='Average Hourly Load (2024-04-01 to 2024-05-31)')
scatter = px.scatter(df_agg, x='hour_formatted', y='avg_p_current_load_mw', color='avg_p_current_load_mw', color_continuous_scale='Viridis')

for trace in scatter.data:
    fig.add_trace(trace)

fig.update_layout(
    xaxis_title='Hour of Day',
    yaxis_title='Average Load (MW)',
    xaxis=dict(
        categoryorder='array',
        categoryarray=[f'{i:02d}:00' for i in range(24)],  # Ensures all hours are displayed in order
        tickmode='linear',
        showgrid=True,  # Ensure the grid is shown
        gridcolor='lightgray'  # Set gridline color
    ),
    yaxis=dict(
        showgrid=True,  # Ensure the grid is shown
        gridcolor='lightgray',  # Set gridline color
        range=[0, 3.7],  # Set the min and max range for the y-axis
    ),
    width=1500,
    height=800,
    coloraxis_colorbar=dict(
        title="Average Load (MW)"
    ),
    plot_bgcolor='white',  # Set the plot background color to white
    paper_bgcolor='white'  # Set the paper background color to white
)

fig.show()

## Sampled drift data with required fields - (draft)

In [20]:
client = bigquery.Client()

query = (
f'''
    
WITH filtered_data AS (
  SELECT
    TIMESTAMP_TRUNC(event_publish_time, SECOND) AS event_time,
    TIMESTAMP_TRUNC(event_publish_time, HOUR) AS event_hour,
    event_publish_time,
    metrics_metered_active_power_w AS p_active_load,
    metrics_metered_current_load_w AS p_current_load,
    metrics_required_min_power_w AS min_power_w,
    metrics_required_max_power_w AS max_power_w,
    metrics_frequency,
    ROW_NUMBER() OVER (PARTITION BY TIMESTAMP_TRUNC(event_publish_time, SECOND) ORDER BY event_publish_time) AS rn
  FROM
    `fever-data-prod.fever.control_log`
  WHERE
    event_publish_time > "2024-07-10 00:00:00.000000+00:00"
    --AND event_publish_time < "2024-06-04 00:00:00.000000+00:00"
    -- TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 3 DAY)
    AND asset_id = "018c2a0d-7dab-40dc-9c3f-b7e6e62c1bb9"
),

sampled_data AS (
  SELECT
    event_time,
    event_hour,
    AVG(p_current_load) AS avg_p_current_load,
    AVG(metrics_frequency) AS avg_grid_freq,
    MAX(p_current_load) AS p_max,
    MIN(p_current_load) AS p_min
  FROM
    filtered_data
  GROUP BY 
    event_time, event_hour
),

hourly_data AS (
  SELECT
    event_hour,
    MAX(p_current_load) AS p_max_hour,
    MIN(p_current_load) AS p_min_hour
  FROM
    filtered_data
  GROUP BY 
    event_hour
),

accepted_cap AS (
  SELECT
    hour,
    min_load,
    accepted_capacity_watt
  FROM
    `fever-data-prod.greenely.backend_capacity_prediction_performance`
  WHERE
    asset_id = "018c2a0d-7dab-40dc-9c3f-b7e6e62c1bb9"
),

predictions AS (
  SELECT
    TIMESTAMP_TRUNC(prediction_at, HOUR) AS prediction_hour,
    load_to_bid
  FROM
    `fever-data-prod.greenely.d1_prediction`
  WHERE
    bidding_zone = 'SE4'
)

SELECT
  -- GENERAL DATA --
  sd.event_time AS DateTime,
  -- CalcBaseline (SKIPPED)
  p.load_to_bid AS ForecAcPow,
  hd.p_max_hour AS Pmax,
  hd.p_min_hour AS Pmin,

  -- FCR DATA
  avg_grid_freq AS GridFreq,
  1 AS ContStatus_FcrdUp ,
  sd.avg_p_current_load AS Cap_FcrdUp,
  p.load_to_bid AS ForecCapFcrdUp,

  -- accepted Capacity
  ac.min_load AS temp_min_load_for_hour,
  ac.accepted_capacity_watt  AS temp_accepted_capacity_watt
FROM
  sampled_data sd
INNER JOIN
  hourly_data hd
ON
  sd.event_hour = hd.event_hour
INNER JOIN
  predictions p
ON
  sd.event_hour = p.prediction_hour
INNER JOIN
  accepted_cap ac
ON
  sd.event_hour = ac.hour 
ORDER BY
  sd.event_time DESC;
  
''')


query_job = client.query(query)
df_series_avg = query_job.to_dataframe()

In [21]:
df_series_avg.head()

,DateTime,ForecAcPow,Pmax,Pmin,GridFreq,ContStatus_FcrdUp,Cap_FcrdUp,ForecCapFcrdUp,temp_min_load_for_hour,temp_accepted_capacity_watt
0,2024-08-21 11:00:52+00:00,0.0,104614.618372,97304.61843,50.044101,1,104614.618372,0.0,97304.61843,0
1,2024-08-21 11:00:51+00:00,0.0,104614.618372,97304.61843,50.045364,1,104614.618372,0.0,97304.61843,0
2,2024-08-21 11:00:50+00:00,0.0,104614.618372,97304.61843,50.046546,1,104614.618372,0.0,97304.61843,0
3,2024-08-21 11:00:49+00:00,0.0,104614.618372,97304.61843,50.046333,1,104614.618372,0.0,97304.61843,0
4,2024-08-21 11:00:48+00:00,0.0,104614.618372,97304.61843,50.047818,1,104614.618372,0.0,97304.61843,0


In [ ]:
fig =  go.Figure()

fig.add_trace(go.Scatter(x=df_series_avg['DateTime'], y=df_series_avg['Pmax'], name="Pmax"))
fig.add_trace(go.Scatter(x=df_series_avg['DateTime'], y=df_series_avg['Pmin'], name="Pmin"))
#fig.add_trace(go.Scatter(x=df_series_avg['DateTime'], y=df_series_avg['p_max_second'], name="p_max_second"))
#fig.add_trace(go.Scatter(x=df_series_avg['DateTime'], y=df_series_avg['p_min_second'], name="p_min_second"))
fig.add_trace(go.Scatter(x=df_series_avg['DateTime'], y=df_series_avg['Cap_FcrdUp'], name="Cap_FcrdUp"))
fig.add_trace(go.Scatter(x=df_series_avg['DateTime'], y=df_series_avg['ForecCapFcrdUp'], name="ForecCapFcrdUp"))

fig.update_layout(
    title="Load and Predictions (Greenely, S3)",
    yaxis={
        'title': {'text': 'Load (W)', 'font': {'size': 16}},
        'overlaying':'y'
    },
    xaxis={
        'title': {'text': 'Date', 'font': {'size': 16}},
        'overlaying':"x",
        
    },
    autosize=True,
    hovermode = 'x',
    width=2000,
    height=750)

fig.show()

## Full drift data for 2 months: 20240401-20240531

In [37]:
client = bigquery.Client()

query = (
f'''
    
WITH filtered_data AS (
  SELECT
    event_publish_time AS event_time,
    TIMESTAMP_TRUNC(event_publish_time, HOUR) AS event_hour,
    event_publish_time,
    metrics_metered_active_power_w AS p_active_load,
    metrics_metered_current_load_w AS p_current_load,
    metrics_required_min_power_w AS p_min,
    metrics_required_max_power_w AS p_max,
    metrics_frequency AS grid_freq,
    ROW_NUMBER() OVER (PARTITION BY TIMESTAMP_TRUNC(event_publish_time, SECOND) ORDER BY event_publish_time) AS rn
  FROM
    `fever-data-prod.fever.control_log`
  WHERE
    event_publish_time > "2024-04-01 00:00:00.000000+00:00"
    AND event_publish_time < "2024-06-01 00:00:00.000000+00:00"
    --event_publish_time > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 3 DAY)
    AND asset_id = "018c2a0d-7dab-40dc-9c3f-b7e6e62c1bb9"
  QUALIFY rn = 1
),

sampled_data AS (
  SELECT
    event_time,
    event_hour,
    grid_freq,
    p_current_load,
    p_min,
    p_max,
  FROM
    filtered_data
),

accepted_cap AS (
  SELECT
    hour,
    min_load,
    accepted_capacity_watt
  FROM
    `fever-data-prod.greenely.backend_capacity_prediction_performance`
  WHERE
    asset_id = "98b95ea6-16e8-45de-83b1-f5676d373e8d"
),

predictions AS (
  SELECT
    TIMESTAMP_TRUNC(prediction_at, HOUR) AS prediction_hour,
    load_to_bid
  FROM
    `fever-data-prod.greenely.d1_prediction`
  WHERE
    bidding_zone = 'SE4'
)

SELECT
  -- GENERAL DATA --
  sd.event_time AS DateTime,
  NULL AS CalcBaseline,
  ROUND(p.load_to_bid / 1000000.0, 4) AS ForecAcPow,
  ROUND(sd.p_current_load / 1000000.0, 4) AS Pmax,
  0 AS Pmin,
  NULL AS LERUp,
  NULL AS LERDown,

  -- FCR DATA
  ROUND(grid_freq, 3) AS GridFreq,
  NULL AS ContStatus_Fcrn,
  1 AS ContStatus_FcrdUp,
  NULL AS ContStatus_FcrdDown,
  NULL AS Cap_Fcrn,
  ROUND(sd.p_current_load / 1000000.0, 4) AS Cap_FcrdUp,
  NULL AS Cap_FcrdDown,
  NULL AS ForecCapFcrn,
  NULL AS ProcuCapFcrn,
  ROUND(p.load_to_bid / 1000000.0, 4) AS ForecCapFcrdUp,
  NULL AS ProcuCapFcrdUp,
  NULL AS ForecCapFcrdDown,
  NULL AS ProcuCapFcrdDown,
FROM
  sampled_data sd
INNER JOIN
  predictions p
ON
  sd.event_hour = p.prediction_hour
INNER JOIN
  accepted_cap ac
ON
  sd.event_hour = ac.hour 
ORDER BY
  sd.event_time DESC;

''')


query_job = client.query(query)
df_series = query_job.to_dataframe()

In [39]:
df_series.to_csv("FE_EV_SE4_2_FCR-D_SE4_20240401T0000-20240531T2359_1s_20240601.csv", index=False)

## Live observation test

In [15]:
client = bigquery.Client()

query = (
f'''
    
WITH filtered_data AS (
  SELECT
    event_publish_time AS event_time,
    TIMESTAMP_TRUNC(event_publish_time, HOUR) AS event_hour,
    event_publish_time,
    metrics_metered_active_power_w AS p_active_load,
    metrics_metered_current_load_w AS p_current_load,
    metrics_required_min_power_w AS p_min,
    metrics_required_max_power_w AS p_max,
    metrics_frequency AS grid_freq,
    ROW_NUMBER() OVER (PARTITION BY TIMESTAMP_TRUNC(event_publish_time, SECOND) ORDER BY event_publish_time) AS rn
  FROM
    `fever-data-prod.fever.control_log`
  WHERE
    event_publish_time > '2024-07-23 18:00:00.0000+00:00'
    AND event_publish_time < '2024-07-23 20:00:00.000000+00:00'
    AND asset_id = '018c2a0d-7dab-40dc-9c3f-b7e6e62c1bb9'
  QUALIFY rn = 1
),

sampled_data AS (
  SELECT
    event_time,
    event_hour,
    grid_freq,
    p_current_load,
    p_min,
    p_max,
  FROM
    filtered_data
)

SELECT
  sd.event_time AS DateTime,
  ROUND(sd.p_current_load / 1000000.0, 4) AS InsAcPow,
  ROUND(grid_freq, 3) AS GridFreq,
  NULL AS RefAcPow
FROM
  sampled_data sd
ORDER BY
  sd.event_time DESC;

''')


query_job = client.query(query)
df_series = query_job.to_dataframe()

In [16]:
df_series.head()

,DateTime,InsAcPow,GridFreq,RefAcPow
0,2024-07-23 19:59:59.074000+00:00,0.1434,50.008,<NA>
1,2024-07-23 19:59:58.095000+00:00,0.1434,50.009,<NA>
2,2024-07-23 19:59:57.086000+00:00,0.1434,50.008,<NA>
3,2024-07-23 19:59:56.067000+00:00,0.1434,50.011,<NA>
4,2024-07-23 19:59:55.074000+00:00,0.1434,50.011,<NA>


In [ ]:
df_series.to_csv("FE_EV_SE3_2_LiveObservationTest_UTC.csv", index=False)

In [21]:
df_series.to_csv("FE_EV_SE3_2_LiveObservationTest_UTC.csv", index=False)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_series['DateTime'], y=df_series['GridFreq'], name="Frequency (Hz)", yaxis="y2"))
fig.add_trace(go.Scatter(x=df_series['DateTime'], y=df_series['InsAcPow'], name="Charger Load (MW)"))

fig.update_layout(
        title={
        'text': 'Live Observation Test',
        'font': {'size': 30},
        'x': 0.475,  # Center title
        'xanchor': 'center'
    },
    xaxis={
        'title': {'text': 'Time', 'font': {'size': 20}},
        'tickmode': 'linear',
        'dtick': 5*60*1000,  
        'tickformat': '%H:%M',  
        'tickangle': -90,  
        'tickfont': {'size': 14}  
    },
    yaxis={
        'title': {'text': 'Charger Load (MW)', 'font': {'size': 20}},
        'range': [-0.3, 0.3],
        'dtick': 0.1,
        'tickfont': {'size': 14}  
    },
    yaxis2={
        'title': {'text': 'Frequency (Hz)', 'font': {'size': 20}},
        'overlaying': 'y',
        'side': 'right',
        'range': [49.4, 50.6],
        'dtick': 0.1,
        'tickfont': {'size': 14} 
    },
    autosize=False,
    hovermode='x',
    width=1800,
    height=1000
)

fig.show()

## Accuracy of our hourly predictions for 2 months

In [6]:
client = bigquery.Client()

query = (
f'''
    
WITH filtered_data AS (
  SELECT
    event_publish_time AS event_time,
    TIMESTAMP_TRUNC(event_publish_time, HOUR) AS event_hour,
    metrics_metered_active_power_w AS p_active_load,
    metrics_metered_current_load_w AS p_current_load,
    metrics_required_min_power_w AS p_min,
    metrics_required_max_power_w AS p_max,
    metrics_frequency AS grid_freq,
    ROW_NUMBER() OVER (PARTITION BY TIMESTAMP_TRUNC(event_publish_time, SECOND) ORDER BY event_publish_time) AS rn
  FROM
    `fever-data-prod.fever.control_log`
  WHERE
    event_publish_time >= "2024-04-01 00:00:00.000000+00:00"
    AND event_publish_time < "2024-06-01 00:00:00.000000+00:00"
    AND asset_id = "018c2a0d-7dab-40dc-9c3f-b7e6e62c1bb9"
  QUALIFY rn = 1
),
sampled_data AS (
  SELECT
    event_time,
    event_hour,
    grid_freq,
    p_current_load,
    p_min,
    p_max
  FROM
    filtered_data
),
accepted_cap AS (
  SELECT
    hour,
    min_load,
    accepted_capacity_watt
  FROM
    `fever-data-prod.greenely.backend_capacity_prediction_performance`
  WHERE
    asset_id = "018c2a0d-7dab-40dc-9c3f-b7e6e62c1bb9"
),
predictions AS (
  SELECT
    TIMESTAMP_TRUNC(prediction_at, HOUR) AS prediction_hour,
    load_to_bid
  FROM
    `fever-data-prod.greenely.d1_prediction`
  WHERE
    bidding_zone = 'SE4'
),
hourly_metrics AS (
  SELECT
    sd.event_hour,
    p.load_to_bid AS predicted_load,
    MIN(sd.p_current_load) AS min_actual_load
  FROM
    sampled_data sd
  INNER JOIN
    predictions p
  ON
    sd.event_hour = p.prediction_hour
  GROUP BY
    sd.event_hour, p.load_to_bid
),
accuracy_check AS (
  SELECT
    event_hour,
    predicted_load,
    min_actual_load,
    CASE WHEN min_actual_load < predicted_load THEN 0 ELSE 1 END AS prediction_correct
  FROM
    hourly_metrics
),
daily_metrics AS (
  SELECT
    DATE(event_hour) AS Date,
    COUNT(*) AS total_predictions,
    SUM(prediction_correct) AS correct_predictions,
    ROUND(SUM(prediction_correct) / COUNT(*) * 100, 2) AS percentage_correct_predictions
  FROM
    accuracy_check
  GROUP BY
    DATE(event_hour)
)
SELECT
  Date,
  total_predictions,
  correct_predictions,
  percentage_correct_predictions
FROM
  daily_metrics
ORDER BY
  Date DESC
''')


query_job = client.query(query)
df_preds = query_job.to_dataframe()

In [11]:
df_preds.head(10)

,Date,total_predictions,correct_predictions,percentage_correct_predictions
0,2024-05-31,24,24,100.00
1,2024-05-30,24,22,91.67
2,2024-05-29,24,24,100.00
3,2024-05-28,24,24,100.00
4,2024-05-27,24,24,100.00
5,2024-05-26,24,24,100.00
6,2024-05-25,24,24,100.00
7,2024-05-24,24,24,100.00
8,2024-05-23,24,23,95.83
9,2024-05-22,24,24,100.00


In [10]:
df_preds.percentage_correct_predictions.mean()

96.10639344262295

## Maximal observed effect per charger

In [12]:
q = '''


SELECT
  "SE4" AS region,
  ds.type,
  ds.external_id,
  ROUND(MAX(ds.charge_power) / 1000, 2) AS max_power_kw,
FROM
  `fever-data-prod.fever.device_state` ds
INNER JOIN
  `fever-data-prod.fever.assets` a
ON
  ds.asset_id = a.id
WHERE
  asset_id = '018c2a0d-7dab-40dc-9c3f-b7e6e62c1bb9'
  AND event_publish_time >= "2024-04-01 00:00:00.000000+00:00"
  AND event_publish_time < "2024-06-01 00:00:00.000000+00:00"
  AND charge_power > 0
GROUP BY
  1,2,3

'''

df_chargers = bq_run(q)

In [13]:
df_chargers.head()

,region,type,external_id,max_power_kw
0,SE4,easee,11326f4a-4c02-4382-bb4b-e0dbc198b856,7.99
1,SE4,easee,11b23733-21c4-49b5-b10a-0e4e5f093a86,3.57
2,SE4,zaptec,11c6f4dd-5bc0-4daa-841b-17491ed89fc9,2.24
3,SE4,easee,1237f8cd-78a5-44b2-8d14-2fab4f0c6d3f,11.27
4,SE4,easee,12c9b8ec-4661-4fea-b530-093a57d1235d,10.96


In [45]:
df_chargers.drop('external_id', axis=1, inplace=True)

In [46]:
df_chargers.to_csv('FE_EV_SE4_CHARGER_METRICS.csv', index=False)

In [ ]:
df_plot = df.groupby(['charger_id', 'charger_type'])['max_power_kw'].mean().reset_index()
fig = px.histogram(df_plot, x='max_power_kw', nbins=100, title="Max Charging Power per Charger (kW)")

fig.update_traces(hovertemplate=f'<b>{"A"}</b>: %{{y}}<br><b>{"B"}</b>: %{{x}}')

fig.update_layout(
    xaxis_title="Charge Power (kW)",
    yaxis_title="Number of Chargers",
    width=1000,
    height=500,
    template='plotly_white',
    title_font=dict(size=20, family='Arial', color='darkblue'),
    xaxis=dict(
        title=dict(font=dict(size=16)),
        tickfont=dict(size=14),
    ),
    yaxis=dict(
        title=dict(font=dict(size=16)),
        tickfont=dict(size=14),
    )
    )

fig.show()

### Import from BQ (metrics for SE3, SE4)

In [15]:
import pandas as pd

data = {
    'bidding_zone': ['SE3', 'SE4'],
    'percentage_above_load_500kw': [55.92, 5.28],
    'percentage_above_pmax_500kw': [64.09, 8.34],
    'hours_above_load_500kw': [818.42, 77.3],
    'hours_above_pmax_500kw': [937.95, 121.99],
    'percentage_above_load_1000kw': [20.3, 0.04],
    'percentage_above_pmax_1000kw': [24.8, 0.07],
    'hours_above_load_1000kw': [297.12, 0.65],
    'hours_above_pmax_1000kw': [362.98, 1.0],
    'percentage_above_load_2000kw': [2.76, 0],
    'percentage_above_pmax_2000kw': [4.24, 0],
    'hours_above_load_2000kw': [40.43, 0.0],
    'hours_above_pmax_2000kw': [62.0, 0.0],
    'avg_load': [674057.63, 184654.76],
    'avg_pmax': [771899.1, 221391.42],
    'median_load': [556972.24, 142756.56],
    'median_pmax': [640918.38, 174377.47]
}

df = pd.DataFrame(data)

# Display the DataFrame
df.transpose()

,0,1
bidding_zone,SE3,SE4
percentage_above_load_500kw,55.92,5.28
percentage_above_pmax_500kw,64.09,8.34
hours_above_load_500kw,818.42,77.3
hours_above_pmax_500kw,937.95,121.99
percentage_above_load_1000kw,20.3,0.04
percentage_above_pmax_1000kw,24.8,0.07
hours_above_load_1000kw,297.12,0.65
hours_above_pmax_1000kw,362.98,1.0
percentage_above_load_2000kw,2.76,0.0
